# Results Log

## Effective Models  

### Species Prediction


## Species Prediction

**New Sampling Method:**  
1) create windows for each gene across all species  
2) select ~80% of genes to be in training data, remaining ~20% to be in training data  
3) determine species with the fewest genes and the corresponding number of windows associated with that species  
4) randomly sample windows for each gene such that the number of total windows represented (# of genes x # of windows) is the same for each species  

### Keras + Tensorflow Model

**S. cerevisiae + C. elegans + Mouse + Humans** (w/o random sequences)

In [3]:
import numpy as np
np.array([[1486, 1030,   32,   35],
       [ 789, 1620,   13,   25],
       [  17,   13, 2274,  159],
       [  31,   38,  323, 2115]])

array([[1486, 1030,   32,   35],
       [ 789, 1620,   13,   25],
       [  17,   13, 2274,  159],
       [  31,   38,  323, 2115]])

**S. cereivisiae + S. boulardii** (w/o random sequences)  
Epochs: 15  
End validation accuracy: ~0.65  

**S. cerevisiae + C. elegans** (w/o random sequences)  
Epochs: 15  
End validation accuracy: ~0.93
checking again...see cEleg.o24099987)  --> see matching FIDDLE results at fiddle.o24099925  (sCer_cEleg_results/exp1_norand_again)  

**S. cerevisiae + S. pombe** (w/o random sequences)  
Epochs: 15  
End validation accuracy: ~0.75  

**S. cerevisiae + C. elegans + Mouse + Human** ("window" limit, w/o random sequences)  
Epochs: 15  
End validation accruacy: ///

### Keras + Tensorflow Model (w/ Dilated Convolution Branch)  

**S. cerevisiae + C. elegans** (w/o random sequences)  
2 regular convolutional layers + 1 dilated convolutional layer (dilation rate: 1,2)  
Epochs: 15  
End Validation Accuracy: ~0.88  

**S. cerevisiae + C. elegans** (w/o random sequences)  
1 regular convolutional layer + 1 dilated convolutional layer (dilation rate: 1,2)  
Epochs: 15  
End Validation Accuracy: ~0.88  

**S. cerevisiae + C. elegans** (w/o random sequences)  
1 regular convolutional layer + 1 dilated convolutional layer (dilation rate: 1,3)  
Epochs: 15  
End Validation Accuracy: ~0.85 (signs of overfitting after several epochs)  

**S. cerevisiae + C. elegans** (w/o random sequences)  
3 regular convolutional layers + 3 dilated convolution layers (dilation rate: (1,2), (1,4), (1,8))  
Epochs: 15  
End Validation Accuracy: ~0.91  

### Keras Model  

**S. cerevisiae + C. elegans**  
Epochs: 15  
Validation Accuracy: ~0.93  

### FIDDLE Model (Original)  
**Encoder**: 2 convolutional layers (regular convolution) - FIDDLE default  
**Decoder**: 1 convolutional layer - FIDDLE default w/ different final FC layer and output activation functions for classification  


**S. cerevisiae + C. elegans + Random**  
Epochs: 5  
End Validation Accuracy: ~0.6 (keep in mind...baseline = 0.33)  
Notes: cross entropy appeared to still be decreasing at Epoch 5 -> should try more epochs  

**S. cerevisiae + C. elegans**  
Epochs: 5  
End Validation Accuracy: ~0.85 (baseline = 0.5)  

**S. cerevisiae + S. boulardii**  
Epochs: 5  
End Validation Accuracy: ~0.62  

### FIDDLE Model (Dilated Convolution)  

**Encoder:** 2 convolutional layers (dilated convolution at 1st convolutional layer; otherwise FIDDLE default)  
**Decoder:** 1 convolutional layer - FIDDLE default w/ different final FC layer and output activation functions for classification  

**S. cerevisiae + S. boulardii**  
Epochs: 5  
End Validation Accuracy: ~0.6

**S. cerevisiae + S. pombe** (w/o random sequences)  
Epochs: 5  
End Validation Accuracy: ~0.8 (sCer_sPom/exp1_norand)  

### FIDDLE Model (2-Branch Regular + Dilated Convolution)

**Encoder:** 1st branch w/ 2 regular convolutional layers, 2nd branch w/ 1 dilated convolution layer  
Note: The 1st and 2nd branch outputs are stacked (similar to what's done in the 1st step of assembling the decoder) and then this stacked output is flattened and fed through a fully-connected layer such that its output resembles the output of using the single-branched approach, i.e. 1 x 500 representatonal layer for each input item.  

**Decoder:** 1 convolutional layer - FIDDLE default w/ different final FC layer and output activation functions for classification  

## GO Prediction

### Tensorflow + Keras Model  

**S. cerevisiae: response to stress vs. cell cycle** (w/ randomly generated sequences in training AND validation!)    
3 Convolutional Layers  
Epochs: 15  
End Validation (Binary) Accuracy: ~0.8 (baseline: ~0.62, from setting all values to zero)  
End Exact Match Accuracy: ~0.62 (baseline: ~0.33, by guessing "cell cycle")   

**S. cerevisiae: response to stress vs. cell cycle** (w/ randomly generated sequences in training ONLY)    
3 Convolutional Layers  
Epochs: 15  
End Validation (Binary) Accuracy: ~0.65 (baseline: ~0.63, from setting all values to zero)  
End Exact Match Accuracy: ~0.35 (baseline: ~0.55, by guessing "cell cycle")   

----

**S. cerevisiae: response to stress vs. cell cycle** (w/o randomly generated sequences)  
3 Convolutional Layers  
Epochs: 15  
End Validation (Binary) Accuracy: ~0.55 (baseline: N/A given that we only have 2 GO terms to predict)  
End Exact Match Accuracy: ~0.40 (baseline: ~0.58, by guessing "cell cycle")  

**S. cerevisiae: response to stress vs. cell cycle** (w/o randomly generated sequences)  
6 Convolutional Layers  
Epochs: 15  
End Validation (Binary) Accuracy: ~0.55 (baseline: N/A given that we only have 2 GO terms to predict)  
End Exact Match Accuracy: ~0.40 (baseline: ~0.58, by guessing "cell cycle")  

-----

**Mouse + Human: response to oxidative stress, response to hypoxia, response to endoplasmic reticulum stress, cellular response to starvation**  
2 Convolutional Layers  
Epochs: 40 (small datasize)  
End Validation (Binary) Accuracy: ~0.66  
End Exact Match Accuracy: ~0.18  



In [3]:
# indices correspond to order of GO terms listed above  
np.array([[ 98, 196, 294,  49],
       [147, 637, 294, 196],
       [ 49, 637, 196, 147],
       [  0, 392,  49, 147]])

array([[ 98, 196, 294,  49],
       [147, 637, 294, 196],
       [ 49, 637, 196, 147],
       [  0, 392,  49, 147]])

**Mouse + Human: response to oxidative stress, response to hypoxia, response to endoplasmic reticulum stress, cellular response to starvation**  
3 Convolutional Layers  
Epochs: 30 (small datasize)  
End Validation (Binary) Accuracy: TBD  
End Exact Match Accuracy: TBD  




**Human: response to oxidative stress, response to hypoxia, response to endoplasmic reticulum stress, cellular response to starvation**  
2 Convolutional Layers  
Epochs: 50 (small datasize)  
End Validation (Binary) Accuracy: ~0.72  
End Exact Match Accuracy: ~0.37  

**Human: chromosome organization (Type II) vs. skeletal system development (Type I)**  
2 Convolutional Layers  
Epochs: 30  
End Validation (Binary) Accuracy:  ~0.54  
End Exact Match Accuracy: ~0.50  

**Human: skeletal system development (Type I) vs. multicellular organism development (Type II)**  
2 Convolutional Layers (Average Pooling)  
Epochs: 30  
End Validation (Binary) Accuracy: ~0.52  
End Exact Match Accuracy: ~0.57  
NOTE: balanced training/validation dataset (many more "skeletal" GO terms actually available)  

**Human: skeletal system development (Type I) vs. multicellular organism development (Type II)**  
2 Convolutional Layers (Max Pooling), Filter Width: 10  
Epochs: 30  
End Validation (Binary) Accuracy: ~0.48   
End Exact Match Accuracy: ~0.46  
NOTE: balanced training/validation dataset (many more "skeletal" GO terms actually available)  

**S. cerevisiae: translation (Type II/TCT) vs. cell cycle

----

### Tensorflow + Keras Model (Multi-Label Binary Classification)  

**Human + Mouse: response to oxidative stress, response to hypoxia, response to endoplasmic reticulum stress, cellular response to starvation**  
2 Convolutional Layers  
Epochs: 30  
End Validation Accuracy:  ~0.67  
End Exact Match Accuracy: ~0.28 (but oxidative stress only, i.e. [1,0,0,0], is ~28% of validation dataset...)  

**S. cerevisiae: cell cycle, stress, chromosome organization**  
2 Convolutional Layers  
Epochs: 15  
End Validation Accuracy: ~0.56  
End Exact Match Accuracy: ~0.20  

**S. cerevisiae: cell cycle vs. stress**  
2 Convolutional Layers  
Epochs: 15  
End Validation Accuracy: ~0.62  
End Exact Match Accuracy: ~0.47  

**S. cerevisiae: cell cycle vs. stress vs. chromosome organization (test only on cell cycle)**  
2 Convolutional Layers  
Epochs: 20  
End Validation/Exact Match Accuracy: ~0.56  

**Human: skeletal system development (Type I) vs. multicellular organism development (Type II)**  
2 Convolutional Layers (Average Pooling), Filter Width: 10  
Epochs: 20  
End Validation (Binary) Accuracy: // 
End Exact Match Accuracy: //   
NOTE: balanced training/validation dataset (many more "skeletal" GO terms actually available)  

